In [84]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.core.display import display, HTML


import pandas as pd
import numpy as np

In [85]:
death_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
confirmed_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
recovered_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
country_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/web-data/data/cases_country.csv')

In [86]:
death_df.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,8/31/21,9/1/21,9/2/21,9/3/21,9/4/21,9/5/21,9/6/21,9/7/21,9/8/21,9/9/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,7118,7123,7127,7127,7127,7127,7141,7144,7151,7157
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,2498,2501,2505,2508,2512,2515,2519,2523,2528,2531
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,5269,5302,5339,5373,5399,5420,5445,5471,5489,5519
3,NaN,Andorra,42.50630,1.521800,0,0,0,0,0,0,...,130,130,130,130,130,130,130,130,130,130
4,NaN,Angola,-11.20270,17.873900,0,0,0,0,0,0,...,1217,1227,1235,1248,1258,1270,1282,1299,1309,1313


In [87]:
confirmed_df.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,8/31/21,9/1/21,9/2/21,9/3/21,9/4/21,9/5/21,9/6/21,9/7/21,9/8/21,9/9/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,153220,153260,153306,153375,153375,153375,153534,153626,153736,153840
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,146387,147369,148222,149117,150101,150997,151499,152239,153318,154316
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,196080,196527,196915,197308,197659,198004,198313,198645,198962,199275
3,NaN,Andorra,42.50630,1.521800,0,0,0,0,0,0,...,15033,15046,15052,15055,15055,15055,15069,15070,15070,15078
4,NaN,Angola,-11.20270,17.873900,0,0,0,0,0,0,...,47544,47781,48004,48261,48475,48656,48790,49114,49349,49628


In [88]:
country_df.columns

Index(['Country_Region', 'Last_Update', 'Lat', 'Long_', 'Confirmed', 'Deaths',
       'Recovered', 'Active', 'Incident_Rate', 'People_Tested',
       'People_Hospitalized', 'Mortality_Rate', 'UID', 'ISO3', 'Cases_28_Days',
       'Deaths_28_Days'],
      dtype='object')

In [89]:
#data cleaning - renaming
country_df.columns = map(str.lower, country_df.columns)
death_df.columns = map(str.lower, death_df.columns)
confirmed_df.columns = map(str.lower, confirmed_df.columns)
recovered_df.columns = map(str.lower, recovered_df.columns)

In [90]:
#renameing columns
confirmed_df = confirmed_df.rename(columns = {'province/state': 'state','country/region':'country'})
country_df = country_df.rename(columns = {'country_region':'country'})
death_df = death_df.rename(columns = {'province/state': 'state','country/region':'country'})
recovered_df = recovered_df.rename(columns = {'province/state': 'state','country/region':'country'})

In [91]:
country_df.dtypes

country                 object
last_update             object
lat                    float64
long_                  float64
confirmed                int64
deaths                   int64
recovered              float64
active                 float64
incident_rate          float64
people_tested          float64
people_hospitalized    float64
mortality_rate         float64
uid                      int64
iso3                    object
cases_28_days            int64
deaths_28_days           int64
dtype: object

In [92]:
sorted_country_df = country_df.sort_values('confirmed', ascending= False)

In [93]:
sorted_country_df.head()

,country,last_update,lat,long_,confirmed,deaths,recovered,active,incident_rate,people_tested,people_hospitalized,mortality_rate,uid,iso3,cases_28_days,deaths_28_days
181,US,2021-09-10 19:21:41,40.000000,-100.00000,40696406,655875,NaN,NaN,12352.221790,NaN,NaN,1.611629,840,USA,4294964,35283
79,India,2021-09-10 19:21:41,20.593684,78.96288,33174954,442009,NaN,NaN,2403.974535,NaN,NaN,1.332358,356,IND,1057128,11755
23,Brazil,2021-09-10 19:21:41,-14.235000,-51.92530,20958899,585174,NaN,NaN,9860.254645,NaN,NaN,2.792007,76,BRA,673832,18278
185,United Kingdom,2021-09-10 19:21:41,55.000000,-3.00000,7201962,134313,NaN,NaN,10608.905482,NaN,NaN,1.864950,826,GBR,956292,3150
144,Russia,2021-09-10 19:21:41,61.524000,105.31880,7000636,187768,NaN,NaN,4797.109607,NaN,NaN,2.682156,643,RUS,535399,21798


In [94]:
def highlight_col(x):
    r = 'background-color: red'
    p = 'background-color: purple'
    g = 'background-color: grey'
    temp_df = pd.DataFrame('',index=x.index, columns= x.columns)
    temp_df.iloc[:,4] = p
    temp_df.iloc[:,5] = r
    temp_df.iloc[:,6] = g
    return temp_df
    
sorted_country_df.style.apply(highlight_col, axis =None)

,country,last_update,lat,long_,confirmed,deaths,recovered,active,incident_rate,people_tested,people_hospitalized,mortality_rate,uid,iso3,cases_28_days,deaths_28_days
181,US,2021-09-10 19:21:41,40.000000,-100.000000,40696406,655875,nan,nan,12352.221790,nan,nan,1.611629,840,USA,4294964,35283
79,India,2021-09-10 19:21:41,20.593684,78.962880,33174954,442009,nan,nan,2403.974535,nan,nan,1.332358,356,IND,1057128,11755
23,Brazil,2021-09-10 19:21:41,-14.235000,-51.925300,20958899,585174,nan,nan,9860.254645,nan,nan,2.792007,76,BRA,673832,18278
185,United Kingdom,2021-09-10 19:21:41,55.000000,-3.000000,7201962,134313,nan,nan,10608.905482,nan,nan,1.864950,826,GBR,956292,3150
144,Russia,2021-09-10 19:21:41,61.524000,105.318800,7000636,187768,nan,nan,4797.109607,nan,nan,2.682156,643,RUS,535399,21798
62,France,2021-09-10 19:21:41,46.227600,2.213700,6972934,116036,nan,nan,10682.639537,nan,nan,1.664091,250,FRA,492879,3207
180,Turkey,2021-09-10 19:21:41,38.963700,35.243300,6590384,59170,nan,nan,7814.153315,nan,nan,0.897823,792,TUR,571929,6467
81,Iran,2021-09-10 19:21:41,32.427908,53.688046,5258913,113380,nan,nan,6261.135979,nan,nan,2.155959,364,IRN,917533,16720
6,Argentina,2021-09-10 19:21:41,-38.416100,-63.616700,5218993,113099,nan,nan,11547.523566,nan,nan,2.167066,32,ARG,152740,4530
37,Colombia,2021-09-10 19:21:41,4.570900,-74.297300,4925000,125480,nan,nan,9679.089731,nan,nan,2.547817,170,COL,68405,2383


In [95]:
import plotly.express as px

In [96]:
fig = px.scatter(sorted_country_df.head(10), x= 'country',y='confirmed',size= 'confirmed',color='country', hover_name='country',size_max=60)
fig.update_layout()
fig.show()

In [97]:
import plotly.graph_objects as go

def plot_cases_of_a_country(country):
    labels = ['confirmed', 'deaths']
    colors = ['blue', 'red']
    mode_size = [6, 8]
    line_size = [4, 5]
    
    df_list = [confirmed_df, death_df]
    
    fig = go.Figure();
    
    for i, df in enumerate(df_list):
        if country == 'World' or country == 'world':
            x_data = np.array(list(df.iloc[:, 20:].columns))
            y_data = np.sum(np.asarray(df.iloc[:,4:]),axis = 0)
            
        else:    
            x_data = np.array(list(df.iloc[:, 20:].columns))
            y_data = np.sum(np.asarray(df[df['country'] == country].iloc[:,20:]),axis = 0)
        print(i)
        fig.add_trace(go.Scatter(x=x_data, y=y_data, mode='lines+markers',
        name=labels[i],
        line=dict(color=colors[i], width=line_size[i]),
        connectgaps=True,
        text = "Total " + str(labels[i]) +": "+ str(y_data[-1])
        ));
    
    fig.update_layout(
        title="COVID 19 cases of " + country,
        xaxis_title='Date',
        yaxis_title='No. of Confirmed Cases',
        margin=dict(l=20, r=20, t=40, b=20),
        paper_bgcolor="lightgrey",
        width = 800,
        
    );
    
    fig.update_yaxes(type="linear")
    fig.show();


plot_cases_of_a_country('US')
#interact(plot_cases_of_a_country,country='world');

0
1


In [98]:
interact(plot_cases_of_a_country, country='World')

0
1


<function __main__.plot_cases_of_a_country(country)>

In [99]:
!pip install folium
import folium

In [100]:
confirmed_df.dropna(axis=0,inplace=True)

In [102]:
world_map = folium.Map(location=[11,0], tiles="cartodbpositron", zoom_start=2, max_zoom = 6, min_zoom = 2)


for i in range(0,len(confirmed_df)):
    folium.Circle(
        location=[confirmed_df.iloc[i]['lat'], confirmed_df.iloc[i]['long']],
        fill=True,
        radius=(int((np.log(confirmed_df.iloc[i,-1]+1.00001)))+0.2)*50000,
        color='red',
        fill_color='indigo',
        tooltip = "<div style='margin: 0; background-color: black; color: white;'>"+
                    "<h4 style='text-align:center;font-weight: bold'>"+confirmed_df.iloc[i]['country'] + "</h4>"
                    "<hr style='margin:10px;color: white;'>"+
                    "<ul style='color: white;;list-style-type:circle;align-item:left;padding-left:20px;padding-right:20px'>"+
                        "<li>Confirmed: "+str(confirmed_df.iloc[i,-1])+"</li>"+
                        "<li>Deaths:   "+str(death_df.iloc[i,-1])+"</li>"+
                        "<li>Death Rate: "+ str(np.round(death_df.iloc[i,-1]/(confirmed_df.iloc[i,-1]+1.00001)*100,2))+ "</li>"+
                    "</ul></div>",
        ).add_to(world_map)

world_map

In [103]:
!pip install voila

     |████████████████████████████████| 1.6 MB 35.3 MB/s eta 0:00:01
     |████████████████████████████████| 393 kB 69.8 MB/s eta 0:00:01
     |████████████████████████████████| 77 kB 20.6 MB/s eta 0:00:01
     |████████████████████████████████| 52 kB 5.1 MB/s  eta 0:00:01
